In [1]:
# Dependencies
import json
import requests
import pandas as pd

import csv
import re
import os

# 1) Fetch data from ElasticSearch bulk API using the Index, Timestamp range ,fetch size

In [2]:
# QA Elastic search url
elasticsearch = "https://miesqa.clouddqt.capitalone.com/"

# Index 
index="card_asvpartnerservicing_partnernotification_app-td-000001"
# Operation to perform
operation="/_search"
url=f"{elasticsearch}{index}{operation}"

# Number of records to search in one go
querystring = {"size":"10"}

# Start time for the search
start_ts="2019-08-25T23:58:00.000Z"
# End time for the search
end_ts="2019-08-25T23:58:30.000Z"

data=[]
document_id=[]
logstash_time=[]

# 2) Create a POST Payload using the above filter criterea

In [3]:
payload= {"sort" : [{ "logstash_time" : {"order" : "asc"}},{"_id": "asc"} ],"query": {"bool": {"filter": [{"range": {"@timestamp": {"gte": start_ts,"lte": end_ts}}}]}}}                


# 3) Create the Header for the API

In [4]:
headers = {
    'Content-Type': "application/json"
    }

# 4) Hit the API to get the fetch count records

In [5]:
# Get the response
response = requests.request("POST", url, data=json.dumps(payload) , headers=headers, params=querystring,verify=False)

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# 5) Validate if the retrun record size is lesser than the Fetch size count set

In [6]:
# If the total response if less than the fetch size 

size=int(querystring["size"])
total_hits=response.json()["hits"]["total"]
total_hits
if size > total_hits:
    size = total_hits

# 6) Get the doucment id and sort value for the last record for pagination

In [7]:
# Get the values for the pagination for next set of request to pull
search_val1=response.json()["hits"]["hits"][size-1]["sort"][0]
search_val2=response.json()["hits"]["hits"][size-1]["sort"][1]
total_hits=response.json()["hits"]["total"]

# 7) Get the next records till the end of the total size fetched

In [8]:
i=size
while i <= total_hits:        
        j=0
        search_val1=response.json()["hits"]["hits"][size-1]["sort"][0]
        search_val2=response.json()["hits"]["hits"][size-1]["sort"][1]
        while j < size and response.json()["hits"]["hits"][j]["_source"]["message"]:
            document_id.append(response.json()["hits"]["hits"][j]["_id"])
            data.append(response.json()["hits"]["hits"][j]["_source"]["message"])
            j=j+1
        payload= {"search_after": [search_val1,search_val2],"sort" : [{ "logstash_time" : {"order" : "asc"}},{"_id": "asc"} ],"query": {"bool": {"filter": [{"range": {"@timestamp": {"gte": start_ts,"lte": end_ts}}}]}}}                
        response = requests.request("POST", url, data=json.dumps(payload), headers=headers, params=querystring,verify=False)
        size=len(response.json()["hits"]["hits"])
        if size == 0:
            break;
        i=i + size;
        
    

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# 8) Put the data into a dataframe and convert into CSV

In [9]:
kibana_output = pd.DataFrame(data,document_id) 
kibana_output.reset_index()
kibana_output.to_csv("./outputData/logData.csv")

# 9) Analyse the above created logs file and create a training data for sagemaker

In [10]:

with open('./outputData/Training.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerow(['SequenceNumber','IsPan','bin','sixteendigits','fifteendigits','fourteendigits','startwithfive','startwithfour'])
    
    

# 10) Analyse the log data for  
# a) If Bin if present 
# b) If Bin Yes - If it has 14,15 or 16 digits 
# c) If digits present check if it starts with 5,4

In [2]:
# Import the list of bins
from binConfig import searchstrList

bin_found=False
# sequence = 0
with open('./outputData/logData.csv','r') as file:
    for line in file:
#         print(line.split(",")[0])  
        for s in searchstrList:
            
            if s in line:
                bin_found=True
                index=line.find(s)
                if index >=0:
  
                    dataitem19=(line[index:index+19])
                    dataitem16=(line[index:index+16])
                    dataitem15=(line[index:index+15])
                    dataitem14=(line[index:index+14])
                    
                    dataList=[]
                    dataList.insert(0,line.split(",")[0])

                    dataList.insert(2,s)
                    if dataitem16.isnumeric()== True:
                        dataList.insert(3,1)    
                    else:
                        dataList.insert(3,0)
                        
                    if dataitem15.isnumeric()== True:
                        dataList.insert(4,1)    
                    else:
                        dataList.insert(4,0)
                        
                    if dataitem14.isnumeric()== True:
                        dataList.insert(5,1)
                        dataList.insert(1,1)        
                    else:
                        dataList.insert(5,0)
                        dataList.insert(1,0)
                    if s[1] == '5':
                        dataList.insert(6,1)    
                    else:
                        dataList.insert(6,0)
                   
                    if s[1] == '4':
                        dataList.insert(7,1)
                    else:
                        dataList.insert(7,0)
                    with open('./outputData/Training.csv', 'a') as csvfile:
                        filewriter = csv.writer(csvfile)
                        filewriter.writerow(dataList)
                        
        if bin_found == False:
            dataList=[]
            dataList.insert(0,line.split(",")[0])
            if line.split(",")[0] == '':
                print("empty doc id")
            else:
                dataList.insert(1,0)    
#                 dataList.insert(2,0)    
                dataList.insert(3,0)    
                dataList.insert(4,0)    
                dataList.insert(5,0)    
                dataList.insert(6,0)    
                dataList.insert(7,0)   
                with open('./outputData/Training.csv', 'a') as csvfile:
                    filewriter = csv.writer(csvfile)
                    filewriter.writerow(dataList)


empty doc id
